http://geojsonviewer.nsspot.net/

In [ ]:
import pymongo
import json

In [ ]:
# Parámetros de la base de datos
host = 'localhost'
port = '27017'
db_name = 'test'

In [ ]:
def transform_json( element ):
    geo_element = dict()
    geo_element[ 'type'] = 'Feature'
    geo_element[ 'geometry'] = dict()
    
    try:
        geo_element[ 'geometry'][ 'type'] = element[ 'location' ][ 'type' ]
        geo_element[ 'geometry'][ 'coordinates'] = element[ 'location' ][ 'coordinates' ]
    except KeyError:
        geo_element[ 'geometry'][ 'type'] = element[ 'geometry' ][ 'type' ]
        geo_element[ 'geometry'][ 'coordinates'] = element[ 'geometry' ][ 'coordinates' ]
    
    geo_element[ 'properties'] = dict()
    geo_element[ 'properties'][ 'name'] = element[ 'name' ]
    return geo_element

In [ ]:
def create_feature_collection( features ):  
    feature_collection = dict()
    feature_collection[ 'type' ] = 'FeatureCollection'
    feature_collection[ 'features' ] = features
    return feature_collection

In [ ]:
def save_json( data ):
    with open( './data/ouput.json', 'w' ) as outfile:
        json.dump( data, outfile )

In [ ]:
# Conexión a la base de datos
db = None
try:
    db_client = pymongo.MongoClient( host + ':' + port )
    db = db_client[ db_name ]
    print( 'Conecction established successfully!' )
except pymongo.errors.ConnectionFailure:
    print( 'Connection cannot be established' )

In [ ]:
db.restaurants.create_index( [ ( 'location', '2dsphere' ) ] )
db.neighborhoods.create_index( [ ( 'geometry', '2dsphere' ) ] )

In [ ]:
point = db.restaurants.find_one()
point

In [ ]:
geo_point = transform_json( point )
geo_point

In [ ]:
save_json( geo_point )

In [ ]:
polygon = db.neighborhoods.find_one()

In [ ]:
geo_polygon = transform_json( polygon )
geo_polygon

In [ ]:
save_json( geo_polygon )

In [ ]:
intersect = db.neighborhoods.find_one( { 'geometry' : { '$geoIntersects' : { '$geometry' : { 'type' : 'Point', 'coordinates' : [ -73.93414657, 40.82302903 ] } } } } )

In [ ]:
save_json( transform_json( intersect ) )

In [ ]:
neighborhood = db.neighborhoods.find_one( { 'geometry' : { '$geoIntersects' : { '$geometry' : { 'type' : 'Point', 'coordinates' : [ -73.93414657, 40.82302903 ] } } } } )
restaurants = db.restaurants.find( { 'location': { '$geoWithin' : { '$geometry' : neighborhood[ 'geometry' ] } } } )

geo_restaurants = []
for restaurant in restaurants:
    geo_restaurants.append( transform_json( restaurant ) )
geo_feature_collection = create_feature_collection( geo_restaurants )
geo_feature_collection

In [ ]:
save_json( geo_feature_collection )

In [ ]:
restaurants = db.restaurants.find( { 'location' : { '$geoWithin' : { '$centerSphere' : [ [ -73.93414657, 40.82302903 ], 5 / 3963.2 ] } } }) # 5 millas

geo_restaurants = []
for restaurant in restaurants:
    geo_restaurants.append( transform_json( restaurant ) )
save_json( create_feature_collection( geo_restaurants ) )

In [ ]:
restaurants = db.restaurants.find( { 'location' : { '$nearSphere' : { '$geometry' : { 'type' : 'Point', 'coordinates' : [ -73.93414657, 40.82302903 ] }, '$maxDistance' : 5000 } } } )

geo_restaurants = []
for restaurant in restaurants:
    geo_restaurants.append( transform_json( restaurant ) )
save_json( create_feature_collection( geo_restaurants ) )